Загрузка библиотек

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install transformers datasets torch pillow jiwer evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.7 MB/s eta 0:00:00


In [ ]:
from transformers import VisionEncoderDecoderModel, TrOCRProcessor, AutoTokenizer
from transformers import Trainer, TrainingArguments,  DefaultDataCollator
from tqdm import tqdm
import time
from transformers import DataCollatorForSeq2Seq
import torch
from torch.nn.utils.rnn import pad_sequence
from datasets import load_from_disk, concatenate_datasets
import glob
import os
import evaluate
import numpy as np

In [ ]:
model = VisionEncoderDecoderModel.from_pretrained('/content/drive/MyDrive/results/fine_tuned_model')
processor = TrOCRProcessor.from_pretrained('/content/drive/MyDrive/results/fine_tuned_model')
model.to('cuda')

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (i

In [ ]:
!unzip drive/MyDrive/processed_dataset_chunks_test.zip

Archive:  drive/MyDrive/processed_dataset_chunks_test.zip
   creating: processed_dataset_chunks_test/
   creating: processed_dataset_chunks_test/chunk_0000_000000_001544/
  inflating: processed_dataset_chunks_test/chunk_0000_000000_001544/data-00000-of-00006.arrow  
  inflating: processed_dataset_chunks_test/chunk_0000_000000_001544/data-00001-of-00006.arrow  
  inflating: processed_dataset_chunks_test/chunk_0000_000000_001544/data-00002-of-00006.arrow  
  inflating: processed_dataset_chunks_test/chunk_0000_000000_001544/data-00003-of-00006.arrow  
  inflating: processed_dataset_chunks_test/chunk_0000_000000_001544/data-00004-of-00006.arrow  
  inflating: processed_dataset_chunks_test/chunk_0000_000000_001544/data-00005-of-00006.arrow  
  inflating: processed_dataset_chunks_test/chunk_0000_000000_001544/dataset_info.json  
  inflating: processed_dataset_chunks_test/chunk_0000_000000_001544/state.json  


In [ ]:
!unzip drive/MyDrive/processed_dataset_chunks.zip

unzip:  cannot find or open drive/MyDrive/processed_dataset_chunks.zip, drive/MyDrive/processed_dataset_chunks.zip.zip or drive/MyDrive/processed_dataset_chunks.zip.ZIP.


In [ ]:
!unzip drive/MyDrive/остаток.zip

Archive:  drive/MyDrive/остаток.zip
   creating: остаток/
   creating: остаток/chunk_0007_035000_040000/
  inflating: остаток/chunk_0007_035000_040000/data-00000-of-00018.arrow  
  inflating: остаток/chunk_0007_035000_040000/data-00001-of-00018.arrow  
  inflating: остаток/chunk_0007_035000_040000/data-00002-of-00018.arrow  
  inflating: остаток/chunk_0007_035000_040000/data-00003-of-00018.arrow  
  inflating: остаток/chunk_0007_035000_040000/data-00004-of-00018.arrow  
  inflating: остаток/chunk_0007_035000_040000/data-00005-of-00018.arrow  
  inflating: остаток/chunk_0007_035000_040000/data-00006-of-00018.arrow  
  inflating: остаток/chunk_0007_035000_040000/data-00007-of-00018.arrow  
  inflating: остаток/chunk_0007_035000_040000/data-00008-of-00018.arrow  
  inflating: остаток/chunk_0007_035000_040000/data-00009-of-00018.arrow  
  inflating: остаток/chunk_0007_035000_040000/data-00010-of-00018.arrow  
  inflating: остаток/chunk_0007_035000_040000/data-00011-of-00018.arrow  
  infla

Предобработка изображений

In [ ]:
# только при обучении модели от microsoft
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')

# Расширяем токенизатор под кириллицу
tokenizer = processor.tokenizer
cyrillic_chars = list("абвгдеёжзийклмнопрстуфхцчшщъыьэюя" +
                      "АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ")
new_tokens = [ch for ch in cyrillic_chars if ch not in tokenizer.get_vocab()]
tokenizer.add_tokens(new_tokens)
print(f"Добавлено {len(new_tokens)} новых токенов в словарь")

# Ресайз эмбеддингов модели
from transformers import VisionEncoderDecoderModel
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')
model.decoder.resize_token_embeddings(len(tokenizer))

# Обновляем processor, чтобы он использовал новый токенизатор
processor.tokenizer = tokenizer


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Функция предобработки данных

In [ ]:

output_folder_test = 'processed_dataset_chunks_test/'
chunk_dirs_test = glob.glob(os.path.join(output_folder_test, "chunk_*"))
datasets_list_test = [load_from_disk(d) for d in chunk_dirs_test if os.path.isdir(d)]
if datasets_list_test:
  final_dataset_test = concatenate_datasets(datasets_list_test)
  final_dataset_test.set_format(type='torch', columns=['pixel_values', 'labels'])

In [ ]:
final_dataset_test

Dataset({
    features: ['pixel_values', 'labels'],
    num_rows: 1544
})

In [ ]:
output_folder = 'processed_dataset_chunks/'
chunk_dirs = glob.glob(os.path.join(output_folder, "chunk_*"))
datasets_list = [load_from_disk(d) for d in chunk_dirs if os.path.isdir(d)]
if datasets_list:
  final_dataset = concatenate_datasets(datasets_list)
  final_dataset.set_format(type='torch', columns=['pixel_values', 'labels'])

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
output_folder = 'остаток/'
chunk_dirs = glob.glob(os.path.join(output_folder, "chunk_*"))
datasets_list = [load_from_disk(d) for d in chunk_dirs[:-1] if os.path.isdir(d)]
if datasets_list:
  final_dataset = concatenate_datasets(datasets_list)
  final_dataset.set_format(type='torch', columns=['pixel_values', 'labels'])

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
# Применение предобработки к датасету
processed_dataset = final_dataset


# Проверка словаря токенизатора
vocab = processor.tokenizer.get_vocab()
print("Размер словаря токенизатора:", len(vocab))
print("Первые 50 элементов словаря:")
print(list(vocab.items())[:50])

# Проверка токенизации текста
sample_texts = ["привет", "зарегистрирован", "тест на русском"]
for text in sample_texts:
    tokens = processor.tokenizer(text, return_tensors="pt").input_ids
    decoded_text = processor.tokenizer.batch_decode(tokens, skip_special_tokens=True)[0]
    print(f"\nИсходный текст: {text}")
    print(f"Токены: {tokens.tolist()[0]}")
    print(f"Декодированный текст: {decoded_text}")

Размер словаря токенизатора: 50331
Первые 50 элементов словаря:
[('Ġgravy', 33729), ('edition', 31855), ('Ġcomes', 606), ('Ġeuro', 2287), ('ĠForce', 3177), ('Ġgorilla', 37692), ('Ġhips', 28097), ('Ġuttered', 36385), ('ĠOp', 7286), ('Either', 40671), ('Ġrhet', 44235), ('Ġdetailed', 4271), ('Ġgrieving', 22567), ('ĠPop', 7975), ('Ġwithdrawing', 20049), ('Ġbeware', 36798), ('ĠPiano', 34802), ('Ġè£ıç', 50079), ('ĠCedar', 15875), ('Ġheritage', 7788), ('Ġbiomass', 36782), ('ĠPhilippe', 16478), ('dress', 36220), ('ĠRED', 24428), ('Ġfear', 2490), ('Ġspacious', 23975), ('Ġaccomplished', 9370), ('\\\\\\\\', 49964), ('Ġhit', 478), ('Ġeagerly', 23908), ('ĠMRI', 20398), ('Ġrounds', 5509), ('timeout', 49109), ('ĠAmbassador', 9175), ('social', 19027), ('ĠMaker', 27253), ('ĠKE', 37112), ('Ġsuppliers', 7593), ('Ġnobility', 45560), ('.""', 39732), ('Ġconsulted', 18998), ('ĠChau', 27619), ('ĠEmacs', 49365), ('school', 8813), ('only', 8338), ('annabin', 45123), ('hatt', 16001), ('Ġbeneficiaries', 14956), (

In [ ]:
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
print('len(tokenizer)=', len(processor.tokenizer))
print('decoder vocab_size=', model.decoder.config.vocab_size)
#должны быть одинаковы

len(tokenizer)= 50331
decoder vocab_size= 50331


In [ ]:
ids = processed_dataset[0]['labels']
print(processor.tokenizer.decode([i for i in ids if i != -100], skip_special_tokens=True))
# должен нормально декодироваться

совокупность


Дообучение модели

In [ ]:
def collate_fn(batch):
    pixel_values = torch.stack([example["pixel_values"] for example in batch])
    labels = [torch.tensor(example["labels"]) for example in batch]

    # Паддим labels -100
    labels = pad_sequence(labels, batch_first=True, padding_value=-100)

    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
cer_metric = evaluate.load("cer")
wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # убираем -100, чтобы корректно декодировать
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer, "wer": wer}

In [ ]:

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/results',
    num_train_epochs=5, # было 7
    per_device_train_batch_size=8,
    logging_dir='./logs',
    remove_unused_columns=True,
    fp16=True,  # Смешанная точность
    eval_strategy="steps",  # или "epoch"
    eval_steps=500,
    load_best_model_at_end=True, # Загрузка лучшей модели по валидации
    metric_for_best_model="eval_loss", # Метрика для выбора лучшей модели
    greater_is_better=False,
    save_strategy="steps",
    save_steps=20000,
    save_total_limit=1,
    logging_steps=500,
)


# # Шаг 5: Функция для поиска последней контрольной точки
# def get_last_checkpoint(output_dir):
#     checkpoints = [d for d in os.listdir(output_dir) if d.startswith("checkpoint-")]
#     if not checkpoints:
#         return None
#     last_checkpoint = max(checkpoints, key=lambda x: int(x.split("-")[1]))
#     return os.path.join(output_dir, last_checkpoint)

# output_dir = '/content/drive/MyDrive/results'

# # Шаг 6: Загрузка модели (с контрольной точки, если есть)
# last_checkpoint = get_last_checkpoint(output_dir)
# if last_checkpoint:
#     print(f"Возобновление обучения с {last_checkpoint}")
#     model = VisionEncoderDecoderModel.from_pretrained(last_checkpoint)
# else:
#   print("Нет контрольных точек")
# Настройка конфигурации модели
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.vocab_size = model.config.decoder.vocab_size

# Шаг 8: Создание Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    data_collator=collate_fn,
    eval_dataset = final_dataset_test,
)

trainer.train()

In [ ]:
# Шаг 9: Сохранение финальной модели
model.save_pretrained('/content/drive/MyDrive/results/fine_tuned_model')
processor.save_pretrained('/content/drive/MyDrive/results/fine_tuned_model')

[]

In [ ]:
import gc
import torch
# Очистка памяти
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from PIL import Image
model.eval()
with torch.no_grad():
  for i in range(65):
    image = Image.open(f'test{i}.png').convert('RGB')
    pixel_values = processor(image, return_tensors='pt').pixel_values.to('cuda')
    generated_ids = model.generate(pixel_values)
    text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(text)

ибо
осталось
оператор
Попробум
Определим
найти
будем
доказать
выдеем
Пусть
поле
новое
разремино
15.09.2020
оператор
Обозианим
расимотим
Предполочим
сверху
отаждестллтььс
более мелкие
нушно
нормальной
они могут
Авторизация
для проведени
кормальная
Покашем
на  онованим
матриуа
что
ели
собствлиий
Покашем
пересталовочи
ковуго
Услови
мошно
полу-
чили
говоричь
дальнейших
ислледваний
надо,
Для
разностная
схема
уравнение
функуии
явная сеема
сдвит
неявная
иисто
вопрое
Знаненяями
стоочной
нижний чтолок
условие
правыи
ря
Аналог
погрешнооьь
раснладпвать
будем
условия
обознание
функция
со значением


FileNotFoundError: [Errno 2] No such file or directory: 'test68.png'

In [ ]:
model.eval()
with torch.no_grad():
  image = Image.open(f'aa1.png').convert('RGB')
  pixel_values = processor(image, return_tensors='pt').pixel_values.to('cuda')
  generated_ids = model.generate(pixel_values)
  text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
  print(text)

NameError: name 'Image' is not defined

AttributeError: 'TrOCRProcessor' object has no attribute 'to'

In [ ]:
import evaluate
import numpy as np

cer_metric = evaluate.load("cer")
wer_metric = evaluate.load("wer")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from torch.utils.data import DataLoader
model.eval()
test_loader = DataLoader(final_dataset_test, batch_size=32, collate_fn=collate_fn)
all_preds, all_refs = [], []
with torch.no_grad():
  for batch in test_loader:
    pixel_values = batch["pixel_values"].to('cuda')
    labels = batch['labels']

    generated_ids = model.generate(pixel_values, max_length=128)
    pred_texts = processor.batch_decode(generated_ids, skip_special_tokens=True)
    labels = np.where(labels.numpy() != -100, labels.numpy(), processor.tokenizer.pad_token_id)
    ref_texts = processor.batch_decode(labels, skip_special_tokens=True)

    all_preds.extend(pred_texts)
    all_refs.extend(ref_texts)


/tmp/ipython-input-639209638.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = [torch.tensor(example["labels"]) for example in batch]


In [ ]:
cer = cer_metric.compute(predictions=all_preds, references=all_refs)
wer = wer_metric.compute(predictions=all_preds, references=all_refs)

In [ ]:
print(f"CER: {cer:.4f}")
print(f"WER: {wer:.4f}")

CER: 0.1984
WER: 0.6439
